In [29]:
# J'importe les modules nécessaires pour le script.
import requests  # Pour effectuer des requêtes HTTP.
from bs4 import BeautifulSoup  # Pour parser le contenu HTML.
import pandas as pd  # Pour manipuler et stocker les données sous forme de tableaux.
import os  # Pour interagir avec le système de fichiers de l'ordinateur.
import time  # Pour introduire des délais dans l'exécution du script.
import random  # Pour générer des nombres aléatoires.

# Read the CSV file containing species IDs and image IDs
csv_file_path = './dataset-mushroom-observer/qualified-dataset/all_ids.csv'
df = pd.read_csv(csv_file_path)
# df = df_full.head(1) # use only first one for testing
display(df.head())

,Overall_ID,Merged_IDs
0,1174,"405904,414778,594982,806590,336937,619647,1156..."


In [ ]:
# Je définis les en-têtes HTTP pour simuler une requête d'un navigateur web moderne.
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Je spécifie le chemin du dossier où les images téléchargées seront enregistrées.
img_folder_path = 'D:/LAYER0/MO/MO'

# Je vérifie si le dossier spécifié existe et sinon, je le crée.
if not os.path.exists(img_folder_path):
    os.makedirs(img_folder_path)

# J'initialise une session de requêtes HTTP pour maintenir les cookies et autres en-têtes à travers les requêtes.
session = requests.Session()
session.headers.update(headers)  # J'ajoute les en-têtes définis à la session.

# Je fixe l'URL de base du site que je souhaite scraper.
base_url = 'https://mushroomobserver.org/'

# Je prépare une liste pour stocker les données extraites.
data = []

for index, row in df.iterrows():
    species_id = row['Overall_ID']
    image_ids = row['Merged_IDs'].split(',')  
    
    for image_id in image_ids:
        page_url = f'{base_url}{image_id}'
        url_reference = image_id

        sleep_time = random.randint(5, 10)
        time.sleep(sleep_time)

        try:
            response = session.get(page_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            name_tag = soup.find('h1', id='title').find('i')
            name = name_tag.get_text(strip=True) if name_tag else f'Unknown_{image_id}'

            img_tags = soup.find_all('img', class_='carousel-image')
            img_counter = 1
            for img_tag in img_tags:
                img_url = img_tag.get('data-src')
                if img_url:
                    img_data = session.get(img_url).content
                    img_name = f"{url_reference}_{name.replace(' ', '_')}_{img_counter}.jpg"
                    img_path = os.path.join(img_folder_path, img_name)
                    
                    with open(img_path, 'wb') as f:
                        f.write(img_data)
                    
                    data.append({'URL': page_url, 'Name': name, 'Image': img_name})
                    img_counter += 1
        
        except requests.exceptions.RequestException as e:
            print(f"Error during request {page_url}: {e}")
            continue

df = pd.DataFrame(data)
print(df)

output_path = 'D:/LAYER0/MO/dataset.csv'
df.to_excel(output_path, index=False)

print(f'Data saved to {output_path}')